In [1]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir + "/code")

from model.CANnoloAttackDetector import CANnoloAttackDetector
from data_helpers.CANDataset import CANDataset
from helpers import calculate_feature_vec_length, seperate_attack_loader, calculate_metrics
from dotenv import load_dotenv
# from data_helpers.CANDataLoader import CANDataLoader
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)

config = {
    "batch_size": 32,
    "delta_time_last_msg": {
        "specific_to_can_id": False,
        "records_back": 30
    },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 15
    },
}

ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)


Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CANData' structure
No attack labels in accelerator_attack_drive_1.log
No attack labels in accelerator_attack_drive_2.log
No attack labels in accelerator_attack_reverse_1.log
No attack labels in accelerator_attack_reverse_2.log
Found attack labels in correlated_signal_attack_1.log
Found attack labels in correlated_signal_attack_2.log
Found attack labels in correlated_signal_attack_3.log
Found attack labels in fuzzing_attack_1.log
Found attack labels in fuzzing_attack_2.log
Found attack labels in fuzzing_attack_3.log
Found attack labels in max_engine_coolant_temp_attack.log
Found attack labels in max_speedometer_attack_1.log
Found attack labels in max_speedometer_attack_2.log
Found attack labels in max_speedometer_attack_3.log
Found attack labels in reverse_light_off_attack_1.log
Found attack labels in rever

In [2]:
display(attack_loaders)

In [3]:

unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len

# Load model
model_path = 'models/canolo_model_112.pt'
threshold = 6.3e-06

model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1, # not sure why + 1 rn but it works
    "feature_vec_length": calculate_feature_vec_length(config)
}

detector = CANnoloAttackDetector(model_path, threshold, model_config)


#     # make enought detectors the length of attack_loaders * deep copy
#     detectors = [deepcopy(detector) for _ in range(len(attack_loaders))]
#     # make a list of tuples of detectors and loaders
#     detector_loader_pairs = zip(detectors, attack_loaders)
#     # run the detectors in parallel
#     results = executor.map(lambda x: calculate_metrics(x[0].detect_attacks(x[1])), detector_loader_pairs)
#     # convert the results to a dictionary
#     results = dict(zip([loader.can_data[0].filename[0] for loader in attack_loaders], results))


MPS is available. Using MPS...


In [4]:
# detector.determine_threshold(ambient_loader)

# 0.0000065
# 0.0000065
# 0.0000065
# 0.0000063
# 0.000165
# .122

# ninetyfive_percentile = .0000156


In [5]:
for loader in attack_loaders:
    # print the column names
    print(loader.can_data[0].actual_attack.sum())
    print(loader.can_data[0].head())

2086
       time   aid              data                        filename  \
0  0.000000  1505  893FC00B0A013880  correlated_signal_attack_1.log   
1  0.000001   651  0000000000000000  correlated_signal_attack_1.log   
2  0.000003   167  0010FA24D12E00A0  correlated_signal_attack_1.log   
3  0.000004   208  4A7704600201F000  correlated_signal_attack_1.log   
4  0.000997    51  000698000E4207D0  correlated_signal_attack_1.log   

   delta_time_last_msg  delta_time_last_same_aid  actual_attack  
0         0.000000e+00                       NaN          False  
1         1.072884e-06                       NaN          False  
2         2.026558e-06                       NaN          False  
3         9.536743e-07                       NaN          False  
4         9.930134e-04                       NaN          False  
2140
           time   aid              data                        filename  \
0  0.000000e+00  1505  891FA0070A00CC80  correlated_signal_attack_2.log   
1  9.536743e-07  

In [6]:
from concurrent.futures import ProcessPoolExecutor
from detect import detect, detect_wrapper

# def detect(loader, detector):
#     filename = loader.can_data[0].filename[0]
#     print(f"Testing on {filename}")
#     return filename, calculate_metrics(detector.detect_attacks(loader))

# def detect_wrapper(args):
#     return detect(args[0], args[1])

thresholds = [.000095, .000085, .000075, .000065, .000055]
meta_data = {}
for threshold in thresholds:
    num_attack_files = len(attack_loaders)
    detectors = []

    for _ in range(num_attack_files):
        detector = CANnoloAttackDetector(model_path, threshold, model_config, force_cpu=True)
        detectors.append(detector)
    
    with ProcessPoolExecutor() as executor:
        meta_data[threshold] = list(executor.map(detect_wrapper, zip(attack_loaders, detectors)))

    for loader in attack_loaders:
        loader.reset()

    print(f"Finished testing on threshold: {threshold}")
    print(meta_data[threshold])



MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...
MPS is available. Using MPS...


RuntimeError: _share_filename_: only available on CPU

In [4]:
detector.threshold = 5.2644566676462956e-05


In [5]:
from tqdm import tqdm

meta_results = {}
for threshold in [.000095, .000085, .000075, .000065, .000055]:
    results = {}
    for i, loader in enumerate(attack_loaders):
        filename = loader.can_data[0].filename[0]
        print(f"Testing on {filename}")
        results[filename] = calculate_metrics(detector.detect_attacks(loader))
        print(results[filename])
    meta_results[threshold] = results

print(meta_results)

Testing on correlated_signal_attack_1.log


100%|█████████▉| 2381/2382 [00:29<00:00, 80.92it/s]


(0.9597595548089038, 0.0, 0.0, 0, [[0, 980], [2086, 73126]])
Testing on correlated_signal_attack_2.log


100%|█████████▉| 2042/2043 [00:23<00:00, 85.12it/s]


(0.9562469392752204, 0.0, 0.0, 0, [[0, 720], [2139, 62485]])
Testing on correlated_signal_attack_3.log


...

(0.9767574376199616, 0.0, 0.0, 0, [[0, 2206], [2444, 195414]])
Testing on max_speedometer_attack_2.log


100%|█████████▉| 4276/4277 [01:05<00:00, 65.76it/s]


(0.9710301683816651, 0.0, 0.0, 0, [[0, 824], [3140, 132868]])
Testing on max_speedometer_attack_3.log


 83%|████████▎ | 5195/6263 [01:33<00:19, 55.43it/s]


KeyboardInterrupt: 